In [1]:
from general_tools.notebook import gpu_utils
GPU = 0
gpu_utils.setup_one_gpu(GPU)

Picking GPU 0


In [22]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt

from tf_lab.fundamentals.utils import reset_tf_graph
import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec

from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, match_incomplete_to_complete_data

from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch, \
                                           plot_interpolations

from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs
                                                  

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

from tf_lab.nips.shape_net import pc_loader as sn_pc_loader
from tf_lab.autopredictors.exploration import latent_embedding_of_entire_dataset
from tf_lab.autopredictors.scripts.virt_scan_data import load_partial_pointclouds

from tf_lab.autopredictors.exploration import find_neighbors

In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [6]:
# Load Complete Shapes

In [11]:
n_pc_samples = 2048
loss = 'chamfer'

In [9]:
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
full_pclouds_path = osp.join(top_data_dir, 'Point_Clouds/Shape_Net/Core/from_manifold_meshes/centered/', str(n_pc_samples))
full_file_names = pio.load_filenames_of_input_data(full_pclouds_path, '.ply')
full_pclouds, full_model_ids, full_syn_ids = pio.load_crude_point_clouds(file_names=full_file_names,\
                                                                           n_threads=50, loader=sn_pc_loader)
print '%d files containing complete point clouds were found.' % (len(full_pclouds), )
model_unames = full_model_ids + '.' + full_syn_ids

57449 files containing complete point clouds were found.


/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/tf_lab/point_clouds/in_out.py:56: UserWarning: Point clouds with the same model name were loaded.
  warnings.warn('Point clouds with the same model name were loaded.')


In [10]:
# Load autoencoder & extract latent codes.

In [13]:
reset_tf_graph()
full_net_name = '1_three_fc_dec_spn_' + str(n_pc_samples) +  'pts_' + loss
full_net_dir = osp.join(top_data_dir, 'OUT/models/nips/vanilla_ae', full_net_name)
full_conf = Conf.load(osp.join(full_net_dir, 'configuration'))

full_ae = PointNetAutoEncoder(full_net_name, full_conf)
saved_epochs = read_saved_epochs(full_conf.train_dir)
last_epoch = saved_epochs[-1]
full_ae.restore_model(full_conf.train_dir, last_epoch, verbose=True)

full_data = PointCloudDataSet(full_pclouds, labels=model_unames, init_shuffle=False)
temp_pclouds, full_latent, temp_model_ids = latent_embedding_of_entire_dataset(full_data, full_ae, full_conf)
assert(np.all(temp_pclouds==full_pclouds))

Model restored in epoch 1000.


In [23]:
indices, distances = find_neighbors(full_latent, k=4)

In [24]:
import bpy

ImportError: No module named bpy

In [ ]:
full_pclouds_path
bpy.ops.import_scene.obj(filepath=modelPath)